In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

In [3]:
milePost = pd.read_csv("../DataFiles/milePostFile.csv")
df = pd.read_excel("../DataFiles/RcrackModeldataset.xlsx")

In [4]:
milePost.head()

,X,Y,MP_VALUE,SR,UID,LONGITUDE,LATITUDE,HAE,REF_DATE,REGION,DIRECTION,SOURCE,AB,DISTANCE,COMMENTS
0,-122.169366,47.978215,1,2,2621,-122.169366,47.978215,-56.14,4/21/05,NW,i,1,A,99999.99,NaN
1,-122.167960,47.978424,1,2,2621,-122.167960,47.978424,-53.83,4/21/05,NW,d,1,A,99999.99,NaN
2,-122.147555,47.978414,2,2,6976,-122.147555,47.978414,-53.63,4/21/05,NW,d,1,A,99999.99,NaN
3,-122.133338,47.970205,3,2,16568,-122.133338,47.970205,-41.38,4/21/05,NW,i,1,A,99999.99,NaN
4,-122.132785,47.970380,3,2,16568,-122.132785,47.970380,-1.71,4/21/05,NW,d,1,A,99999.99,NaN


In [5]:
coord = milePost[['X', 'Y', 'MP_VALUE', 'SR']]

In [50]:
coord.dtypes

X           float64
Y           float64
MP_VALUE      int64
SR           object
sr            int64
dtype: object

In [36]:
type(list(coord[['SR']][1:2])[0])

str

In [43]:
def change2ints(row):
    sr = row['SR']
    if len(sr) > 3:
        return 0
    return int(sr)

In [48]:
coord.loc[:,'sr'] = coord.apply(change2ints, axis = 1)

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [49]:
coord.head()

,X,Y,MP_VALUE,SR,sr
0,-122.169366,47.978215,1,2,2
1,-122.167960,47.978424,1,2,2
2,-122.147555,47.978414,2,2,2
3,-122.133338,47.970205,3,2,2
4,-122.132785,47.970380,3,2,2


In [51]:
test = coord[((coord.loc[:,'MP_VALUE'] > 1.0) & (coord.loc[:,'MP_VALUE'] < 3)) & (coord.loc[:,'sr'] < 10)]

In [57]:
db = coord[['X', 'Y', 'MP_VALUE', 'sr']]

In [59]:
db = db[db.loc[:,'sr']>0]  

In [60]:
db.head()

,X,Y,MP_VALUE,sr
0,-122.169366,47.978215,1,2
1,-122.167960,47.978424,1,2
2,-122.147555,47.978414,2,2
3,-122.133338,47.970205,3,2
4,-122.132785,47.970380,3,2


In [89]:
test = np.array(db[((db.loc[:,'MP_VALUE'] > 1.0) & (db.loc[:,'MP_VALUE'] < 3)) & (db.loc[:,'sr'] < 10)][0:1])

In [94]:
test[0][0:2][0]

-122.14755500000001

In [97]:
# df.dtypes

In [98]:
def getLat(row):
    sr = row['SR']
    bmp = int(row['Begin ARM'])+1
    emp = int(row['End ARM'])+1
    res = np.array(db[((db.loc[:,'MP_VALUE'] >= bmp) & (db.loc[:,'MP_VALUE'] <= emp)) & (db.loc[:,'sr'] == sr)])
    if len(res) > 0:
        coordinates = res[0][0:2]
        x = coordinates[0]
        return coordinates[1]
    return None

In [99]:
df['Latitude'] = df.apply(getLat, axis = 1)

In [101]:
def getLong(row):
    sr = row['SR']
    bmp = int(row['Begin ARM'])+1
    emp = int(row['End ARM'])+1
    res = np.array(db[((db.loc[:,'MP_VALUE'] >= bmp) & (db.loc[:,'MP_VALUE'] <= emp)) & (db.loc[:,'sr'] == sr)])
    if len(res) > 0:
        coordinates = res[0][0:2]
        return coordinates[0]
#         return coordinates[1]
    return None

In [102]:
df['Longitude'] = df.apply(getLong, axis = 1)

In [103]:
df.head()

,YearC,Sealed Alligator,Sealed Longitudinal,Int ID,SR,Begin ARM,End ARM,Year,Contract Type Description,Surface Type Code,...,Long,CatLong,AlliP,LongP,Traffic Year,ESALs,Contract History ID,STcode,Latitude,Longitude
0,2003,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,10,L,1.988636,0.946970,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA,46.242524,-123.886017
1,2004,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,84,L,0.946970,7.954545,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA,46.242524,-123.886017
2,2005,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,10,L,1.988636,0.946970,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA,46.242524,-123.886017
3,2006,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,5,L,1.988636,0.473485,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA,46.242524,-123.886017
4,2007,NaN,NaN,54964,101,0.53,0.63,1996,RESURFACE EXISTING RDWY,AA,...,5,L,NaN,0.473485,2013,32740,0073c3c3-1b5a-40af-a002-022c09a2875d,AA,46.242524,-123.886017


In [104]:
df.to_excel("../DataFiles/wholeDataset.xlsx")

In [105]:
df.columns 

Index([                     u'YearC',           u'Sealed Alligator',
              u'Sealed Longitudinal',                     u'Int ID',
                               u'SR',                  u'Begin ARM',
                          u'End ARM',                       u'Year',
        u'Contract Type Description',          u'Surface Type Code',
         u'Surface Type Description',          u'Surface Thickness',
                           u'Region',                       u'ID_y',
                u'Construction Year',           u'Major Rehab Year',
               u'Segment Lane Miles',    u'ESALs Since Major Rehab',
                  u'Total Thickness',    u'Total Surface Thickness',
                    u'Pavement Type', u'Construction Pavement Type',
                 u'Major Rehab Type',     u'Prior Major Rehab Year',
           u'Prior Major Rehab Type',                u'Original ID',
                        u'Is Bridge',          u'Lane Surface Type',
                       u'Lane Mile

In [110]:
df2 = df[[                     u'YearC',                           u'Int ID',
                               u'SR',                  u'Begin ARM',
                          u'End ARM',                       u'Year',
                           u'Region',                       u'ID_y',
               u'Segment Lane Miles',    u'ESALs Since Major Rehab',
                 u'Major Rehab Type',     u'Prior Major Rehab Year',
           u'Prior Major Rehab Type',  u'Is Bridge', u'Lane Surface Type',
                       u'Lane Miles',   u'Federal Functional Class',
            u'Avg Construction Date',       u'Avg Major Rehab Date',
                             u'Alli',                    u'CatAlli',
                             u'Long',                    u'CatLong',
                            u'AlliP', u'LongP', u'Traffic Year',                      u'ESALs',
                             u'STcode', u'Longitude', u'Latitude']]

In [111]:
df2.head()

,YearC,Int ID,SR,Begin ARM,End ARM,Year,Region,ID_y,Segment Lane Miles,ESALs Since Major Rehab,...,CatAlli,Long,CatLong,AlliP,LongP,Traffic Year,ESALs,STcode,Longitude,Latitude
0,2003,54964,101,0.53,0.63,1996,Southwest,0073c3c3-1b5a-40af-a002-022c09a2875d,0.2,478649,...,L,10,L,1.988636,0.946970,2013,32740,AA,-123.886017,46.242524
1,2004,54964,101,0.53,0.63,1996,Southwest,0073c3c3-1b5a-40af-a002-022c09a2875d,0.2,478649,...,L,84,L,0.946970,7.954545,2013,32740,AA,-123.886017,46.242524
2,2005,54964,101,0.53,0.63,1996,Southwest,0073c3c3-1b5a-40af-a002-022c09a2875d,0.2,478649,...,L,10,L,1.988636,0.946970,2013,32740,AA,-123.886017,46.242524
3,2006,54964,101,0.53,0.63,1996,Southwest,0073c3c3-1b5a-40af-a002-022c09a2875d,0.2,478649,...,L,5,L,1.988636,0.473485,2013,32740,AA,-123.886017,46.242524
4,2007,54964,101,0.53,0.63,1996,Southwest,0073c3c3-1b5a-40af-a002-022c09a2875d,0.2,478649,...,NaN,5,L,NaN,0.473485,2013,32740,AA,-123.886017,46.242524


In [112]:
df2.to_excel("../DataFiles/cp255Dataset.xlsx")

In [113]:
df3 = df2[df2.loc[:, 'YearC'] == 1999]

In [114]:
len(df3)

1703

In [116]:
df3.to_csv("../DataFiles/cp255Data.csv")

In [118]:
df2 = df3.dropna(axis=0, how='any', thresh=None, subset=[u'Latitude', u'Longitude'], inplace=False)

In [119]:
print len(df2)
print len(df3)

1435
1703


In [120]:
df2.to_csv("../DataFiles/cp255Data.csv")